In [7]:
PREPROCESSED_SEQUENCE_PATH = '/home/bandeja/datasets/samsung-spatial/processed/2022-04-16-12-17-44_round1/'

In [8]:
import cv2
import numpy as np
import os
from pathlib import Path

### Assoсiate color and depth timestamps

In [9]:
def read_folder(path_to_folder: str):
    """
    Reads all images' paths and timestamps from folder
    :param path_to_folder: path to folder to read
    :return: dictionary where keys are timestamps and values are path to image
    """
    files = os.listdir(path_to_folder)
    timestamps = [float(Path(file).stem) for file in files]
    files = [os.path.join(os.path.basename(path_to_folder), x) for x in files]
    timestamp_image_kvp = dict(zip(timestamps, files))
    return timestamp_image_kvp


def associate(
    color_images: dict, depth_images: dict, offset: float, max_difference: float
) -> list:
    """
    Associates color and depth images
    :param color_images: (timestamp, path) KVP for color images
    :param depth_images: (timestamp, path) KVP for depth images
    :param offset: time offset added to the timestamps of the depth images
    :param max_difference: maximally allowed time difference for matching entries
    :return: best matches for color and depth images
    """
    first_keys = np.asarray(list(color_images.keys()))
    second_keys = np.asarray(list(depth_images.keys()))
    best_matches = list()
    for timestamp in first_keys:
        best_match = second_keys[np.argmin(np.abs(second_keys + offset - timestamp))]
        if abs(best_match + offset - timestamp) < max_difference:
            best_matches.append((timestamp, best_match))
    return sorted(best_matches)

def create_association_files(camera_data_path, sync_1_ts, sync_2_ts):
    color_list = read_folder(os.path.join(camera_data_path, 'color'))
    depth_list = read_folder(os.path.join(camera_data_path, 'depth'))
    depth_dense_list = read_folder(os.path.join(camera_data_path, 'depth_dense'))

    matches_depth = associate(color_list, depth_list, 0, 1e-3)
    matches_depth_dense = associate(color_list, depth_dense_list, 0, 1e-3)

    with open(os.path.join(camera_data_path, 'associations.txt'), 'w') as f:
        for (a, b) in matches_depth:
            # Filter timestamps where shaking is performed
            if a > sync_1_ts and a < sync_2_ts:
                f.write('{0} {1} {2} {3}\n'.format(
                    a,
                    color_list[a],
                    b,
                    depth_list[b]
                ))
                
    with open(os.path.join(camera_data_path, 'associations_dense.txt'), 'w') as f:
        for (a, b) in matches_depth_dense:
            # Filter timestamps where shaking is performed
            if a > sync_1_ts and a < sync_2_ts:
                f.write('{0} {1} {2} {3}\n'.format(
                    a,
                    color_list[a],
                    b,
                    depth_dense_list[b]
                ))

In [10]:
camera_folders = [name for name in os.listdir(PREPROCESSED_SEQUENCE_PATH) if os.path.isdir(os.path.join(PREPROCESSED_SEQUENCE_PATH, name))]
camera_folders

['5s', '3m']

In [11]:
for camera_folder in camera_folders:
    create_association_files(os.path.join(PREPROCESSED_SEQUENCE_PATH, camera_folder), 8, 48)

### Create ORB configs

In [12]:
import shutil

for camera_folder in camera_folders:
    one_camera_data_path = os.path.join(PREPROCESSED_SEQUENCE_PATH, camera_folder)
    K_undist = np.load(os.path.join(one_camera_data_path, 'camera_params.npy'))

    new_config_path = os.path.join(one_camera_data_path, 'ORB_one_view_config.yaml')
    shutil.copyfile('configs/ORB_one_view_config.yaml', new_config_path)

    with open(new_config_path, 'r') as f:
        lines = f.readlines()
        lines[10] = 'Camera1.fx: {}\n'.format(K_undist[0, 0])
        lines[11] = 'Camera1.fy: {}\n'.format(K_undist[1, 1])
        lines[12] = 'Camera1.cx: {}\n'.format(K_undist[0, 2])
        lines[13] = 'Camera1.cy: {}\n'.format(K_undist[1, 2])

    with open(new_config_path, 'w') as f:
        f.writelines(lines)